In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd
import unicodedata
import re

In [ ]:
import torchvision

In [ ]:
df_train = pd.read_csv('./data/train_clean.csv')
df_test = pd.read_csv('./data/test_clean.csv')

In [ ]:
df_train.columns

In [ ]:
q1_att = 'q1_stems_no_stops'
q2_att = 'q2_stems_no_stops'

In [ ]:
texts_1=[]
texts_2 =[]
labels = []
for q1,q2,is_dup in zip(df_train[q1_att],df_train[q2_att],df_train.is_duplicate):
    texts_1.append(str(q1).split())
    texts_2.append(str(q2).split())
    labels.append(int(is_dup))

In [ ]:
test_texts_1 =[]
test_texts_2 =[]
test_ids = []
for q1,q2,id_ in zip(df_test[q1_att],df_test[q2_att],df_test.test_id):
    test_texts_1.append(str(q1).split())
    test_texts_2.append(str(q2).split())
    test_ids.append(id_)

In [ ]:
del df_train,df_test

In [ ]:
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300

In [ ]:
from collections import Counter
import itertools
from collections import Counter
import itertools
def build_vocab(sentences, max_vocab=30000):
    word_counts = Counter(itertools.chain(*sentences))
    vocabulary_inv = []
    vocabulary_inv.append("<PAD/>")
    vocabulary_inv.append("<mino/>")
    vocabulary_inv.extend([x[0] for x in word_counts.most_common(max_vocab)])
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]

In [ ]:
import numpy as np
vocabulary, vocabulary_inv = build_vocab(test_texts_1+test_texts_2+texts_1+texts_2, max_vocab=200000)

In [ ]:
import pickle
output = open('./data/embedding/vacabulary_quora_stemd.pkl', 'wb')
pickle.dump(vocabulary,output)
output.close()
# output = open('./processed_data/vocabulary_inv_quora_cle.pkl', 'wb') 
# pickle.dump(vocabulary,output)
# output.close()

In [ ]:
import pickle
with open('./data/embedding/vacabulary_quora_cle.pkl', 'r') as output:
    vocabulary = pickle.load(output)

In [ ]:
len(vocabulary)

In [ ]:
def build_data(ques1,ques2,label,vocabulary):
    mino_idx = vocabulary["<mino/>"]
    data_index = []
    for q1,q2,dup in zip(ques1,ques2,label):
        sentence1 = [vocabulary.get(word, mino_idx) for word in q1]
        sentence2 = [vocabulary.get(word, mino_idx) for word in q2]
        data_index.append([sentence1,sentence2,dup])
    return data_index
data_index = build_data(texts_1,texts_2,labels,vocabulary)

In [ ]:
def build_test_data(ques1,ques2,vocabulary):
    mino_idx = vocabulary["<mino/>"]
    data_index = []
    for q1,q2 in zip(ques1,ques2):
        sentence1 = [vocabulary.get(word, mino_idx) for word in q1]
        sentence2 = [vocabulary.get(word, mino_idx) for word in q2]
        data_index.append([sentence1,sentence2])
    return data_index
test_data_index = build_test_data(test_texts_1,test_texts_2,vocabulary)

In [ ]:
del texts_1,texts_2,test_texts_1,test_texts_2

In [ ]:
import torch.nn.init as init

In [ ]:
from gensim.models import KeyedVectors
import numpy as np
EMBEDDING_FILE = './data/embedding/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE,binary=True)
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
nb_words = min(MAX_NB_WORDS, len(vocabulary))+1
embedding_matrix = np.random.normal(loc=0,scale=0.1,size=[nb_words, EMBEDDING_DIM])
for word, i in vocabulary.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [ ]:
del word2vec

In [ ]:
nodl_train_fea1 = pd.read_csv('./data/train_no_dl_fea1.csv',usecols=['k_q1_q2_intersect',
       'k_q1_freq', 'k_q2_freq'])

In [ ]:
nodl_test_fea1 = pd.read_csv('./data/test_no_dl_fea1.csv',usecols=['k_q1_q2_intersect',
       'k_q1_freq', 'k_q2_freq'])

In [ ]:
leaks = nodl_train_fea1[['k_q1_q2_intersect','k_q1_freq', 'k_q2_freq']]
test_leaks = nodl_test_fea1[['k_q1_q2_intersect','k_q1_freq', 'k_q2_freq']]

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(np.vstack((leaks, test_leaks)))
train_leaks_fea = ss.transform(leaks)
test_leaks_fea = ss.transform(test_leaks)

In [ ]:
from gensim.models import KeyedVectors

# Creating the model
word2vec = KeyedVectors.load_word2vec_format('./data/embedding/wiki.en.vec')

MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
nb_words = min(MAX_NB_WORDS, len(vocabulary))+1
embedding_matrix = np.random.normal(loc=0,scale=0.1,size=[nb_words, EMBEDDING_DIM])
for word, i in vocabulary.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [ ]:
import numpy as np
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
nb_words = min(MAX_NB_WORDS, len(vocabulary))+1
embeddings_index = {}
f = open('./data/embedding/glove.6B.300d.txt')
count = 0
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
embedding_matrix = np.random.normal(loc=0,scale=0.1,size=[nb_words, EMBEDDING_DIM])
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
del embeddings_index

In [ ]:
np.shape(embedding_matrix)[0]

In [ ]:
class biRNN(nn.Module):
    def __init__(self,vocab_size,embedding_size,hidden_size,output_size,n_layers = 1):
        super(biRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.output_size =output_size
        self.embed= nn.Embedding(vocab_size,embedding_size)
        self.gru= nn.GRU(input_size=self.embedding_size,
                            hidden_size=self.hidden_size,
                            num_layers = self.n_layers,
                            batch_first =True,
                            bidirectional = True)
        self.da = 100
        self.r = 100
        self.fc1 = nn.Linear(100*4+100, 1200)
        self.fc2 = nn.Linear(1200, 2)
        self.fc_leak = nn.Linear(3,100)
        self.fcWs1 = nn.Linear(self.hidden_size*2, self.da)
        self.fcWs2 = nn.Linear(self.da,self.r)
        self.fc_dedim_mul = nn.Linear(self.hidden_size*2*self.r,100)
        self.fc_dedim_sub = nn.Linear(self.hidden_size*2*self.r,100)
        self.fc_dedim_rep = nn.Linear(self.hidden_size*2*self.r,100)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
    def forward(self,leak_input,sentence1,sentence2,length1,length2,pad1_V,pad2_V,cuda_available = False):
        
#         h1 = torch.mean(self.embed(sentence1),1)
#         h1 =torch.squeeze(h1, dim=1)
#         h2 = torch.mean(self.embed(sentence2),1)
#         h2 =torch.squeeze(h2,dim=1)

        input_1 = self.embed(sentence1)
        input_2 = self.embed(sentence2)
        if cuda_available:
            output_1 = torch.mul(pad1_V.float().cuda(),input_1)
            output_2 = torch.mul(pad2_V.float().cuda(),input_2)
        else:
            output_1 = torch.mul(pad1_V.float(),input_1)
            output_2 = torch.mul(pad2_V.float(),input_2)
        output_1,hidden1 = self.gru(output_1)
        
        output_2,hidden2 = self.gru(output_2)
#         output_1,hidden1 = self.gru(self.embed(sentence1))
        
#         output_2,hidden2 = self.gru(self.embed(sentence2))
#         if cuda_available:
#             output_1 = torch.mul(pad1_V.float().cuda(),output_1)
#             output_2 = torch.mul(pad2_V.float().cuda(),output_2)
#         else:
#             output_1 = torch.mul(pad1_V.float(),output_1)
#             output_2 = torch.mul(pad2_V.float(),output_2)
#         if cuda_available:
#             output_1_mask = Variable(torch.zeros(output_1.size())).cuda()
#             output_2_mask = Variable(torch.zeros(output_2.size())).cuda()
#         else:
#             output_1_mask = Variable(torch.zeros(output_1.size()))
#             output_2_mask = Variable(torch.zeros(output_2.size()))
#         for i in range(len(length1)):
# #             output1[i] = torch.cat((output_1[i,length1[i]-1,:self.hidden_size],output_1[i,0,-self.hidden_size:]), 0)
# #             output2[i] = torch.cat((output_2[i,length2[i]-1,:self.hidden_size],output_2[i,0,-self.hidden_size:]), 0)
#             A1[i] = torch.mm(self.softmax(torch.t(self.fcWs2(torch.tanh(self.fcWs1(output_1[i,:length1[i],:]))))),
#                                   output_1[i,:length1[i],:])
#             A2[i] = torch.mm(self.softmax(torch.t(self.fcWs2(torch.tanh(self.fcWs1(output_2[i,:length2[i],:]))))),
#                                   output_2[i,:length2[i],:])
        A1 = self.fcWs2(torch.tanh(self.fcWs1(output_1.contiguous().view(-1,self.hidden_size*2)))).view(output_1.size()[0],
                                                                                                        output_1.size()[1],
                                                                                                        -1)
        A1_t = self.softmax(torch.transpose(A1, 1, 2).contiguous().view(-1,output_1.size()[1])).view(output_1.size()[0],-1,output_1.size()[1])
        output1_new = torch.bmm(A1_t, output_1)
        
        A2 = self.fcWs2(torch.tanh(self.fcWs1(output_2.contiguous().view(-1,self.hidden_size*2)))).view(output_2.size()[0],
                                                                                                        output_2.size()[1],
                                                                                                        -1)
        A2_t = self.softmax(torch.transpose(A2, 1, 2).contiguous().view(-1,output_2.size()[1])).view(output_2.size()[0],
                                                                                                     -1,
                                                                                                     output_2.size()[1])
        output2_new = torch.bmm(A2_t, output_2)
    #         A2[i] = torch.mm(self.softmax(torch.t(self.fcWs2(torch.tanh(self.fcWs1(output_2[i,:length2[i],:]))))),
#                                   output_2[i,:length2[i],:])
        output1_new = output1_new.view(-1,self.r*self.hidden_size*2)
        output2_new = output2_new.view(-1,self.r*self.hidden_size*2)
        mul_represent = torch.mul(output1_new,output2_new)
        sub_represent = torch.abs(output1_new-output2_new)
        mul_dedim = self.fc_dedim_mul(mul_represent)
        sub_dedim = self.fc_dedim_sub(sub_represent)
        rep1_dedim = self.fc_dedim_rep(output1_new)
        rep2_dedim = self.fc_dedim_rep(output2_new)
#         mul_represent = torch.mul(rep1_dedim,rep2_dedim)
#         sub_represent = torch.abs(rep1_dedim-rep2_dedim)
        dense_leak = self.fc_leak(leak_input)
        out_mul = self.relu(torch.cat((mul_dedim,sub_dedim,rep1_dedim,rep2_dedim,dense_leak),1))
        logits  = self.fc2(self.relu(self.fc1(out_mul)))
        return logits
    def init_weights(self,pretrained_weight):
        #init(self.embed.weight.data)
        initrange = 0.001
#         nn.init.uniform(self.embedding,a=0,b=0.00
        self.fc1.bias.data.fill_(0)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        #torch.nn.init.xavier_uniform(self.fc.weight.data, gain=math.sqrt(2.0))
        #self.embed.weight.data.normal_(mean=0, std=0.01)
        self.embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.fill_(0)
        
        self.fcWs1.weight.data.uniform_(-initrange, initrange)
        self.fcWs1.bias.data.fill_(0)
        self.fcWs2.weight.data.uniform_(-initrange, initrange)
        self.fcWs2.bias.data.fill_(0)
    def initHidden(self,batch_size):
        h0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
        c0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
        return h0,c0

In [ ]:
# class biRNN(nn.Module):
#     def __init__(self,vocab_size,embedding_size,hidden_size,output_size,n_layers = 1):
#         super(biRNN, self).__init__()
#         self.n_layers = n_layers
#         self.hidden_size = hidden_size
#         self.embedding_size = embedding_size
#         self.output_size =output_size
#         self.embed = nn.Embedding(vocab_size,embedding_size)
#         self.gru = nn.GRU(input_size=self.embedding_size,
#                             hidden_size=self.hidden_size,
#                             num_layers = self.n_layers,
#                             batch_first =True,
#                             bidirectional = True)
#         self.lstm = nn.LSTM(input_size=self.embedding_size,
#                             hidden_size=self.hidden_size,
#                             num_layers = self.n_layers,
#                             batch_first =True,
#                             bidirectional = True)
#         #self.BN0 = nn.BatchNorm1d(2400)
#         self.fc1 = nn.Linear(4*self.hidden_size*2*self.n_layers+100, 1200)
#         #self.BN1 = nn.BatchNorm1d(1200)
#         self.fc2 = nn.Linear(1200, 2)
#         self.fc_leak = nn.Linear(3, 100)
#         self.relu = nn.ReLU()
#         self.softmax = nn.Softmax()
#     def forward(self,leak_input,sentence1,sentence2,length1,length2,pad1_V,pad2_V,cuda_available = False):
# # 
#         input_1 = self.embed(sentence1)
#         input_2 = self.embed(sentence2)
# #         if cuda_available:
# #             output_1 = torch.mul(pad1_V.float().cuda(),input_1)
# #             output_2 = torch.mul(pad2_V.float().cuda(),input_2)
# #         else:
# #             output_1 = torch.mul(pad1_V.float(),input_1)
# #             output_2 = torch.mul(pad2_V.float(),input_2)
#         output_1,hidden1 = self.gru(input_1)
#         output_2,hidden2 = self.gru(input_2)
        
# #         ls_o1,(ls_h1,ls_c1) = self.lstm(input_1)
# #         ls_o2,(ls_h2,ls_c2) = self.lstm(input_2)
        
#         A = torch.transpose(hidden1, 0, 1)
#         B = torch.transpose(hidden2, 0, 1)
#         A = A.contiguous().view(input_1.size()[0],-1)
#         B = B.contiguous().view(input_2.size()[0],-1)
        
# #         A2 = torch.transpose(ls_h1, 0, 1)
# #         B2 = torch.transpose(ls_h2, 0, 1)
# #         A2 = A2.contiguous().view(input_1.size()[0],-1)
# #         B2 = B2.contiguous().view(input_2.size()[0],-1)
# #         output1 = torch.cat((output_1[:,length1,:self.hidden_size],output_1[i,0,-self.hidden_size:]), 0)
# #         output2 = torch.cat((output_2[:,length2,:self.hidden_size],output_2[i,0,-self.hidden_size:]), 0)
# #         A = torch.cat((A1,A2), 1)
# #         B = torch.cat((B1,B2), 1)
#         mul_represent = torch.mul(A,B)
#         sub_represent = torch.abs(A-B)
#         dense_leak = self.fc_leak(leak_input)
#         out_mul = self.relu(torch.cat((mul_represent,sub_represent,A,B,dense_leak),1))
#         logits  = self.fc2(self.relu(self.fc1(out_mul)))
#         return logits
#     def init_weights(self,pretrained_weight):
#         #init(self.embed.weight.data)
#         initrange = 0.001
# #         nn.init.uniform(self.embedding,a=0,b=0.00
#         self.fc1.bias.data.fill_(0)
#         self.fc1.weight.data.uniform_(-initrange, initrange)
#         #torch.nn.init.xavier_uniform(self.fc.weight.data, gain=math.sqrt(2.0))
#         #self.embed.weight.data.normal_(mean=0, std=0.01)
#         self.embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
#         self.fc2.weight.data.uniform_(-initrange, initrange)
#         self.fc2.bias.data.fill_(0)
#     def initHidden(self,batch_size):
#         h0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
#         c0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
#         return h0,c0

In [ ]:
bilstm = biRNN(vocab_size = np.shape(embedding_matrix)[0],embedding_size = np.shape(embedding_matrix)[1],hidden_size = 300,output_size =2)

In [ ]:
bilstm.init_weights(embedding_matrix)

In [ ]:
def get_batch(batch,hidden_num = 100,training = True):
    length1 = [len(sen1) for sen1 in batch[:,0]]
    length2 = [len(sen2) for sen2 in batch[:,1]]
    max_length1 = max(length1)
    max_length2 = max(length2)
    s1_batch = []
    s2_batch = []
    s1_batch_pad = []
    s2_batch_pad = []
    for sen in batch:
        if len(sen[0])==max_length1:
            s1_batch.append(sen[0])
            s1_batch_pad.append(np.concatenate((np.ones([len(sen[0]),hidden_num]),
                                                np.zeros([max_length1-len(sen[0]),hidden_num])),0))
        else:
            s1_batch.append(sen[0]+(max_length1-len(sen[0]))*[0])
            s1_batch_pad.append(np.concatenate((np.ones([len(sen[0]),hidden_num]),
                                                np.zeros([max_length1-len(sen[0]),hidden_num])),0))
        if len(sen[1])==max_length2:
            s2_batch.append(sen[1])
            s2_batch_pad.append(np.concatenate((np.ones([len(sen[1]),hidden_num]),
                                                np.zeros([max_length2-len(sen[1]),hidden_num])),0))
        else:
            s2_batch.append(sen[1]+(max_length2-len(sen[1]))*[0])
            s2_batch_pad.append(np.concatenate((np.ones([len(sen[1]),hidden_num]),
                                                np.zeros([max_length2-len(sen[1]),hidden_num])),0))
    if training:   
        return [Variable(torch.from_numpy(np.array(s1_batch))),
                Variable(torch.from_numpy(np.array(s2_batch))),
                batch[:,2].astype(int),length1,length2,
                Variable(torch.from_numpy(np.array(s1_batch_pad))),
                Variable(torch.from_numpy(np.array(s2_batch_pad)))]
    else:
        return [Variable(torch.from_numpy(np.array(s1_batch))),
                Variable(torch.from_numpy(np.array(s2_batch))),
                length1,length2,
                Variable(torch.from_numpy(np.array(s1_batch_pad))),
                Variable(torch.from_numpy(np.array(s2_batch_pad)))]

In [ ]:
batch_s1,batch_s2,batch_label,length1_b,length2_b ,pad1,pad2= get_batch(np.array(data_index[0:10]),hidden_num=300,training=True)
leak_in = Variable(torch.from_numpy(np.array(train_leaks_fea[0:10]))).float()
o = bilstm(leak_in,batch_s1,batch_s2,length1_b,length2_b,pad1,pad2)

In [ ]:
device_id = 3

In [ ]:
torch.cuda.set_device(device_id)

In [ ]:
learning_rate = 1e-3
criterior_cross = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(bilstm.parameters(), lr=1e-3)
criterior_cross.cuda()
bilstm.cuda()

In [ ]:
def test_model(model_path,trainD,leak_te):
    trainData = trainD.tolist()
    test_leak_data = leak_te.tolist()
    print('len train    '+str(len(trainData)))
    torch_softmax= nn.Softmax()
    bilstm_test = biRNN(vocab_size = np.shape(embedding_matrix)[0],
                        embedding_size = np.shape(embedding_matrix)[1],
                        hidden_size = 300,output_size =2)
    bilstm_test.load_state_dict(torch.load(model_path))
    torch.cuda.set_device(device_id)
    bilstm_test.cuda()
    batch_t_size = 128
    pad_num_train = (len(trainData)/batch_t_size+1)*batch_t_size-len(trainData)
    for i in range(pad_num_train):
        trainData.append([[0,0],[0,0],1])
        test_leak_data.append([0,0,0])
    pred_train = []
    for i in range(len(trainData)/batch_t_size):
        batcht_s1,batcht_s2,batcht_labels,lengtht_s1,lengtht_s2,padt1,padt2 = get_batch(np.array(trainData[i*batch_t_size:(i+1)*batch_t_size]),
                                                                                 hidden_num=300,training=True)
        leak_input = Variable(torch.from_numpy(np.array(test_leak_data[i*batch_t_size:(i+1)*batch_t_size]))).float()
        
        o = bilstm_test(leak_input.cuda(),batcht_s1.cuda(),batcht_s2.cuda(),lengtht_s1,lengtht_s2,padt1,padt2,cuda_available=True)
        out_model = torch_softmax(o)
        pred_train.extend(out_model.data.cpu().numpy()[:,1].astype(float))
        if i%200==0:
            print("train step: "+str(i))
    print('len train pred   '+str(len(pred_train[:-pad_num_train])))
    del trainData
    return pred_train[:-pad_num_train]

def train_model(train_d,dev_d,train_leak,dev_leak,id_m,name_model):
    trainD = np.array(train_d).copy()
    devdata = np.array(dev_d).copy()
    leak_t_D = np.array(train_leak).copy()
    leak_dev_data = np.array(dev_leak).copy()
    batch_size =64
    best_acc = 0
    epoches = 1
    best_loss = 10
    batch_dev_size =128
    torch_softmax= nn.Softmax()
    bilstm_tr = biRNN(vocab_size = np.shape(embedding_matrix)[0],
                   embedding_size = np.shape(embedding_matrix)[1],
                   hidden_size = 300,
                   output_size =2)
    bilstm_tr.init_weights(embedding_matrix)
    crossEn = torch.nn.CrossEntropyLoss()
    optimizer_tr = torch.optim.Adam(bilstm_tr.parameters(), lr=1e-3)
    torch.cuda.set_device(device_id)
    crossEn.cuda()
    bilstm_tr.cuda()
    from sklearn.metrics import log_loss
    path = './model_stack/'+name_model+'0_'+str(id_m)+'.pkl'
    for epoch in range(epoches):
        #perm = np.random.permutation(len(trainD))
        trainData = trainD
        leakdata = leak_t_D
        if epoch==0:
            check_steps=250
        else:
            check_steps=100
        for i in range(len(trainData)/batch_size):
            optimizer_tr.zero_grad()
            batch_s1,batch_s2,batch_labels,length_s1,length_s2,pad1,pad2 = get_batch(np.array(trainData[i*batch_size:(i+1)*batch_size]),
                                                                                     hidden_num=300,training=True)
            leak_input = Variable(torch.from_numpy(np.array(leakdata[i*batch_size:(i+1)*batch_size]))).float()
            if torch.cuda.is_available():
                o_tr = bilstm_tr(leak_input.cuda(),batch_s1.cuda(),batch_s2.cuda(),length_s1,length_s2,pad1,pad2,cuda_available=True)  
            loss_ = crossEn(o_tr, Variable(torch.from_numpy(batch_labels)).cuda())
            loss_.backward()
            optimizer_tr.step()
            if (i+1) % check_steps ==0:
                count_correct =0
                allsum =[]
                pred_dev_batch = []
                true_labels_dev = []
                for num in range(len(devdata)/batch_dev_size):
                    batch_s1_dev,batch_s2_dev,batch_labels_dev,length_s1_dev,length_s2_dev,pad1dev,pad2dev = get_batch(np.array(devdata[num*batch_dev_size:(num+1)*batch_dev_size]),
                                                                                                                       hidden_num=300)
                    leak_dev_input = Variable(torch.from_numpy(np.array(leak_dev_data[num*batch_dev_size:(num+1)*batch_dev_size]))).float()
                    od = bilstm_tr(leak_dev_input.cuda(),batch_s1_dev.cuda(),batch_s2_dev.cuda(),length_s1_dev,length_s2_dev,pad1dev,pad2dev,cuda_available=True)
                    out_model = torch_softmax(od)
                    allsum.append(sum(out_model.data.max(1)[1].view(-1).cpu()))
                    count_correct += sum(out_model.data.max(1)[1].view(-1).cpu().numpy()==batch_labels_dev)#.cuda().long()).long().sum().data[0]
                    pred_dev_batch.extend(out_model.data.cpu().numpy()[:,1].astype(float))
                    #bi_loss = binary_log_loss(o.data[:,1],torch.from_numpy(batch_labels_dev).cuda())
                    true_labels_dev.extend(batch_labels_dev)
                    #print(bi_loss)
                acc = count_correct/(len(devdata)+0.0)
                current_loss = log_loss(true_labels_dev, pred_dev_batch)
#                 if current_loss<best_loss:
#                     best_loss = current_loss
#                     torch.save(bilstm_tr.state_dict(), path)
#                     print("save in : "+ path)
# #                     step = step +1
# #                     if step==num_model:
# #                         step = 0
                print('step: '+str(i) +' acc: '+str(acc)+' '+ str(sum(allsum))+" loss: "+str(current_loss))
    torch.save(bilstm_tr.state_dict(), path)
    print('save in : '+ path)
    return path

In [ ]:
import pickle
with open('./data/kdf.pkl', 'r') as f:
    kf = pickle.load(f)

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_splits=8,random_state=0)
splits = []
train_ids = []
for (train_index, val_index) in kf:
    splits.append([train_index,val_index])
    train_ids.extend(val_index)

In [ ]:
name_ = 'stems_no_stops_self_att_glove'

In [ ]:
from sklearn.metrics import log_loss
train_feature = []
test_feature = []
for num_c,spl in enumerate(splits):
    train_data,val_data = np.array(data_index)[spl[0]],np.array(data_index)[spl[1]]
    train_leaks,dev_leaks = np.array(train_leaks_fea)[spl[0]],np.array(train_leaks_fea)[spl[1]]
    best_model_epo = train_model(train_data,val_data[0:4000],train_leaks,dev_leaks[0:4000],num_c,name_)

In [ ]:
#device_id =2
train_feature = []
for num_c,spl in enumerate(splits):
    path = './model_stack/'+name_+'0_'+str(num_c)+'.pkl'
    val_data = np.array(data_index)[spl[1]]
    dev_leaks = np.array(train_leaks_fea)[spl[1]]
    train_pred = test_model(path,val_data,dev_leaks)
    train_feature.extend(train_pred)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid = np.array(data_index)[:-400],np.array(data_index)[-400:]
leak_train_new_x,leak_valid_new_x = np.array(train_leaks_fea)[:-400],np.array(train_leaks_fea)[-400:]

In [ ]:
model_for_test = train_model(x_train,x_valid,leak_train_new_x,leak_valid_new_x,100,name_+'for_test')

In [ ]:
def test_(model_path,testD,leak_test):
    testData = testD.tolist()
    leak_t = leak_test.tolist()
    print('len test  '+str(len(testData)))
    torch_softmax= nn.Softmax()
    bilstm_test = biRNN(vocab_size = np.shape(embedding_matrix)[0],
                        embedding_size = np.shape(embedding_matrix)[1],
                        hidden_size = 300,output_size =2)
    bilstm_test.load_state_dict(torch.load(model_path))
    torch.cuda.set_device(device_id)
    bilstm_test.cuda()
    batch_t_size = 128
    pad_num_test = (len(testData)/batch_t_size+1)*batch_t_size-len(testData)
    for i in range(pad_num_test):
        testData.append([[0,0],[0,0]])
        leak_t.append([0,0,0])
    pred_test = []
    for i in range(len(testData)/batch_t_size):
        batcht_s1,batcht_s2,lengtht_s1,lengtht_s2,padt1,padt2 = get_batch(np.array(testData[i*batch_t_size:(i+1)*batch_t_size]),
                                                                                 hidden_num=300,training=False)
        leak_input = Variable(torch.from_numpy(np.array(leak_t[i*batch_t_size:(i+1)*batch_t_size]))).float()
        o = bilstm_test(leak_input.cuda(),batcht_s1.cuda(),batcht_s2.cuda(),lengtht_s1,lengtht_s2,padt1,padt2,cuda_available=True)
        out_model = torch_softmax(o)
        pred_test.extend(out_model.data.cpu().numpy()[:,1].astype(float))
        if i%200==0:
            print("test step: "+str(i))      
    print('len test pred  '+str(len(pred_test[:-pad_num_test])))
    del testData
    return pred_test[:-pad_num_test]

In [ ]:
test_pred = test_(model_for_test,np.array(test_data_index),np.array(test_leaks_fea))

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
log_loss(np.array(labels)[train_ids],train_feature)

In [ ]:
train_dl = np.zeros(len(train_ids))
for i,id_ in enumerate(train_ids):
    train_dl[id_] = train_feature[i]

In [ ]:
traindl_file = pd.read_csv('./stack_dl_feature/train_dl_with_leak.csv')

In [ ]:
traindl_file[name_] = train_dl
traindl_file.to_csv('./stack_dl_feature/train_dl_with_leak.csv',index=False)

In [ ]:
testdl_file = pd.read_csv('./stack_dl_feature/test_dl_with_leak.csv')

In [ ]:
testdl_file[name_] = test_pred
testdl_file.to_csv('./stack_dl_feature/test_dl_with_leak.csv',index= False)

In [ ]:
stdmean_file= pd.read_csv('./stack_dl_feature/stdmean_file.csv')
std = np.std(train_dl)
mean = np.mean(train_dl)
print mean,std

In [ ]:
name_

In [ ]:
stdmean_file[name_] = [mean,std]
stdmean_file.to_csv('./stack_dl_feature/stdmean_file.csv',index=False)

In [ ]:
traindl_file = pd.read_csv('./stack_dl_feature/dl_fea_train.csv')

In [ ]:
traindl_file[name_] = train_dl
traindl_file.to_csv('./stack_dl_feature/dl_fea_train.csv',index=False)

In [ ]:
testdl_file = pd.read_csv('./stack_dl_feature/dl_fea_test.csv')

In [ ]:
#pred_dl_f = np.mean(test_feature,axis=0)
testdl_file[name_] = test_pred
testdl_file.to_csv('./stack_dl_feature/dl_fea_test.csv',index= False)